# PANDAS PROJECT - October 24, 2018

## First step, Import the necessary modules

In [1]:
import pandas as pd
import numpy as np
import os
print("Have to install xlrd (with a version equal or higher than 0.9.0)")
print(" >> xlrd module succesfully imported")
print("")
import xlrd
print("To see xlrd version we have to import pkg_resources")
import pkg_resources
print(" >> pkg_resources module succesfully imported")
print("")
print("xlrd version >> "+str(pkg_resources.get_distribution("xlrd").version))

Have to install xlrd (with a version equal or higher than 0.9.0)
 >> xlrd module succesfully imported

To see xlrd version we have to import pkg_resources
 >> pkg_resources module succesfully imported

xlrd version >> 1.1.0


## Database Main Function is to store the evolution of human population for a range of years
## In this case, it begins on 3700 BD and finish on 2000 AD

#### Function to list all files inside a folder tree for a specific file format

In [2]:
def list_files(startpath,form):
    gd=[]
    for root, dirs, files in os.walk(startpath):
        for f in files:
            fn,ext=os.path.splitext(f)
            if ext.lower()==form:
                a=root+"/"+f
                a=a.replace('\\\\','/')
                gd.append(a)
    return gd

## Listing files

In [3]:
gd=list_files(os.getcwd(),'.xlsx')

In [4]:
counter_ind=-1
for item in gd:
    counter_ind+=1
    print("Index "+str(counter_ind)+" >> "+item)

Index 0 >> /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/Week_2/Day_8/pandas-project/your-code/urbanspatial-hist-urban-pop-3700bc-ad2000-xlsx.xlsx


## Reading the Database (Excel File) for the sheet named as "Historical Urban Population"

In [5]:
excel_file=pd.read_excel(gd[0],sheet_name='Historical Urban Population')

In [6]:
print("Displaying the 10 first registers of the Database")
display(excel_file.head(10))

Displaying the 10 first registers of the Database


,Record,City,OtherName,Country,Latitude,Longitude,Certainty,year,pop,cityid
0,1,Eridu,NaN,Iraq,30.815838,45.996070,1,-3700,6000,Eridu_30.8_46
1,2,Eridu,NaN,Iraq,30.815838,45.996070,1,-3500,10000,Eridu_30.8_46
2,3,Larak,Kesh,Iraq,32.312738,45.660998,1,-3500,10000,Larak_32.3_45.7
3,4,Uruk,Erech,Iraq,31.322222,45.636111,1,-3500,14000,Uruk_31.3_45.6
4,5,Anshan,NaN,Iran,30.012020,52.408577,1,-3300,10000,Anshan_30_52.4
5,6,Eridu,NaN,Iraq,30.815838,45.996070,1,-3300,10000,Eridu_30.8_46
6,7,Larak,Kesh,Iraq,32.312738,45.660998,1,-3300,10000,Larak_32.3_45.7
7,8,Uruk,Erech,Iraq,31.322222,45.636111,1,-3300,40000,Uruk_31.3_45.6
8,9,Anshan,NaN,Iran,30.012020,52.408577,1,-3000,10000,Anshan_30_52.4
9,10,Eridu,NaN,Iraq,30.815838,45.996070,1,-3000,10000,Eridu_30.8_46


## Checking basic statistical info and info type

In [7]:
info=excel_file.describe(include='all').T
info['Data Type']=excel_file.dtypes
info=info
display(info)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,Data Type
Record,10352,NaN,NaN,NaN,5176.5,2988.51,1,2588.75,5176.5,7764.25,10352,int64
City,10352,1701,Istanbul,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object
OtherName,4574,715,Soochow,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object
Country,10352,148,China,1166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object
Latitude,10352,NaN,NaN,NaN,34.6748,16.8682,-45.8742,27.3905,37.6257,46.5833,71.2906,float64
Longitude,10352,NaN,NaN,NaN,32.8931,56.6213,-157.858,3.05498,22.8725,76.462,174.776,float64
Certainty,10352,NaN,NaN,NaN,1.24778,0.579281,1,1,1,1,3,int64
year,10352,NaN,NaN,NaN,1511.01,757.905,-3700,1499,1787,1900,2000,int64
pop,10352,NaN,NaN,NaN,285308,939595,0,30000,52423,135000,2.3e+07,int64
cityid,10352,1728,Istanbul_41_29,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object


## Evolution of human population from 3700 BD to 2000 AD

In [8]:
# Spanish Population evolution
pop_spain=excel_file[excel_file['Country']=='Spain'].reset_index(drop=True)

In [9]:
# Which is the oldest country?
# First, lets find the oldest date in the data base
old_country=excel_file.loc[excel_file['year']==excel_file['year'].min()]['Country'][0]
print("")
print(chr(27)+"[1;31m"+"The oldest country registred in the data base >> "+chr(27)+"[1;34m"+old_country+chr(27)+"[0m"+"")
print("")


The oldest country registred in the data base >> Iraq



In [10]:
# Spanish Population evolution
pop_oldest=excel_file[excel_file['Country']==old_country].reset_index(drop=True)

#### Lets check the basics of both data frames

In [11]:
info2=pop_spain.describe(include='all').T
print("")
print(chr(27)+"[1;31m"+">>  Spain Info  <<"+chr(27)+"[0m"+"")
print("")
display(info2)


>>  Spain Info  <<



,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Record,425,NaN,NaN,NaN,4405.9,2448.47,495,2560,3990,6203,10327
City,425,37,Barcelona,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OtherName,73,8,"Cordova, Córdoba",28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,425,1,Spain,425,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Latitude,425,NaN,NaN,NaN,39.0871,2.08085,28.15,37.3881,38.9161,41.3124,43.4647
Longitude,425,NaN,NaN,NaN,-3.06578,2.87527,-16.25,-4.89356,-3.70379,-0.889581,2.65024
Certainty,425,NaN,NaN,NaN,1.08235,0.299833,1,1,1,1,3
year,425,NaN,NaN,NaN,1592.28,315.174,-100,1489,1685,1825,2000
pop,425,NaN,NaN,NaN,123846,398965,2000,24000,40000,70000,4.1e+06
cityid,425,37,Barcelona_41.4_2.2,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
info3=pop_oldest.describe(include='all').T
print("")
print(chr(27)+"[1;31m"+">>  "+old_country.capitalize()+"  <<"+chr(27)+"[0m"+"")
print("")
display(info3)


>>  Iraq  <<



,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Record,230,NaN,NaN,NaN,1377.01,2398.07,1,122.5,291.5,976.25,10072
City,230,34,Baghdad,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OtherName,128,16,Bagdad,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,230,1,Iraq,230,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Latitude,230,NaN,NaN,NaN,32.6726,1.72272,26.1216,31.4148,32.3127,33.3406,36.3594
Longitude,230,NaN,NaN,NaN,45.0854,1.35378,41.9196,44.4009,44.5801,45.8767,50.4989
Certainty,230,NaN,NaN,NaN,1.26957,0.500161,1,1,1,1,3
year,230,NaN,NaN,NaN,-652.191,1690.45,-3700,-2100,-1150,900,2000
pop,230,NaN,NaN,NaN,143504,387753,900,12000,40000,100000,3.841e+06
cityid,230,34,Baghdad_33.3_44.4,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Following function returns year interval and population interval for each city of a country

In [13]:
def dif_pop(df,name_city):
    df_year_pop=df[df['City']==name_city][['year','pop']].sort_values('year',ascending=True).reset_index(drop=True)
    year=df_year_pop['year'].values
    pop=df_year_pop['pop'].values
    if np.size(df_year_pop,0)>1:
        year_interval=year[-1]-year[0]
        return year[0],year[1],pop[0],pop[1]
    else:
        return year[0],year[0],pop[0],pop[0]

#### Spain Cities population evolution evaluation

In [14]:
year_pop_spain=[]
for city_name in pop_spain['City'].unique().tolist():
    y0,y1,p0,p1=dif_pop(pop_spain,city_name)
    year_pop_spain.append([city_name,y0,y1,p0,p1])
    
df_spain=pd.DataFrame(year_pop_spain,columns=['City','from Year','to Year','from Population','to Population'])
df_spain['Population Diff']=df_spain['to Population']-df_spain['from Population']

In [15]:
label=['Decrease','Stable','Increase']

## Defining bins width

margin=[(df_spain['to Population']-df_spain['from Population']).min()*1.1,-10000,10000,
        (df_spain['to Population']-df_spain['from Population']).max()*1.1]

## Decrease will be one with a Population Diff less than -10000 people
## Increase will be one with a Population Diff more than 10000 people
## Stable will be one with a Population Diff between -10000 and 10000 people

In [16]:
bins = pd.cut(df_spain['Population Diff'],margin, labels=label)
final_spain=df_spain
final_spain['Pop Eval']=bins
display(final_spain)

,City,from Year,to Year,from Population,to Population,Population Diff,Pop Eval
0,Cadiz,-100,100,62000,65000,3000,Stable
1,Merida,300,400,100000,100000,0,Stable
2,Seville,622,800,40000,35000,-5000,Stable
3,Cordoba,800,800,160000,160000,0,Stable
4,Toledo,800,1000,25000,31000,6000,Stable
5,Burgos,884,1163,9000,18000,9000,Stable
6,Bobastro,900,900,36000,36000,0,Stable
7,Barcelona,950,1009,2000,9000,7000,Stable
8,Almeria,1000,1300,35000,25000,-10000,Decrease
9,Badajoz,1000,1250,16000,15000,-1000,Stable


In [17]:
spanish_deviation=final_spain['Pop Eval'].value_counts().sort_values()
print("-----------------------------------------------------------------------------")
print("")
print("  >> Summary of Population variation for spanish cities")
display(spanish_deviation)

-----------------------------------------------------------------------------

  >> Summary of Population variation for spanish cities


Decrease     3
Increase     7
Stable      27
Name: Pop Eval, dtype: int64

### Iraq population evolution evaluation (Oldest country available)

In [18]:
year_pop_oldest=[]
for city_name in pop_oldest['City'].unique().tolist():
    y0,y1,p0,p1=dif_pop(pop_oldest,city_name)
    year_pop_oldest.append([city_name,y0,y1,p0,p1])
    
df_oldest=pd.DataFrame(year_pop_oldest,columns=['City','from Year','to Year','from Population','to Population'])
df_oldest['Population Diff']=df_oldest['to Population']-df_oldest['from Population']

In [19]:
label=['Decrease','Stable','Increase']

## Defining bins width

margin=[(df_oldest['to Population']-df_oldest['from Population']).min()*1.1,-10000,10000,
        (df_oldest['to Population']-df_oldest['from Population']).max()*1.1]

## Decrease will be one with a Population Diff less than -10000 people
## Increase will be one with a Population Diff more than 10000 people
## Stable will be one with a Population Diff between -10000 and 10000 people


In [20]:
bins = pd.cut(df_oldest['Population Diff'],margin, labels=label)
final_oldest=df_oldest
final_oldest['Pop Eval']=bins
display(final_oldest)

,City,from Year,to Year,from Population,to Population,Population Diff,Pop Eval
0,Eridu,-3700,-3500,6000,10000,4000,Stable
1,Larak,-3500,-3300,10000,10000,0,Stable
2,Uruk,-3500,-3300,14000,40000,26000,Increase
3,Nippur,-3000,-2800,10000,10000,0,Stable
4,Adab,-2800,-2500,10000,20000,10000,Stable
5,Kish,-2800,-2500,30000,20000,-10000,Decrease
6,Larsa,-2800,-2500,16000,10000,-6000,Stable
7,Shuruppak,-2800,-2500,30000,30000,0,Stable
8,Ur,-2800,-2500,12000,10000,-2000,Stable
9,Zabalam,-2800,-2500,10000,10000,0,Stable


In [21]:
oldest_deviation=final_oldest['Pop Eval'].value_counts().sort_values()
print("-----------------------------------------------------------------------------")
print("")
print("  >> Summary of Population variation for spanish cities")
display(oldest_deviation)

-----------------------------------------------------------------------------

  >> Summary of Population variation for spanish cities


Decrease     4
Increase     5
Stable      25
Name: Pop Eval, dtype: int64

### Concatenating Population summary for both countries

In [22]:
Summ_pop=pd.DataFrame(np.c_[np.r_[spanish_deviation,final_spain['Population Diff'].sum()],
                            np.r_[oldest_deviation,final_oldest['Population Diff'].sum()]])
Summ_pop.columns=['Spain','Iraq']
Summ_pop.index=['Decrease','Increase','Stable','Total Pop variation']
display(Summ_pop)

,Spain,Iraq
Decrease,3,4
Increase,7,5
Stable,27,25
Total Pop variation,340170,507000


# Final Summary

The table below shows us that Spain, Even having more cities in expansion than Iraq from 3700BD to 2000AD,<br>
Iraq has a higher expansion in total population than Spain.<br>
As conclusion, For a deeper study of population evolution we should have an accurate data base with more continued<br>
registers. For each year there are only a couple of registers for each city that make our job more harder than usual<br> 
to reach an overall evaluation of wolrd population with high accuracy.<br>


### Global evaluation (Not totally finished, have to check some values)

In [38]:
%%time

counter=0
for country in excel_file['Country'].unique().tolist():
    counter+=1
    pop_country=excel_file[excel_file['Country']==country]
    year_pop=[]
    for city_name in pop_country['City'].unique().tolist():
        y0,y1,p0,p1=dif_pop(pop_country,city_name)
        year_pop.append([country,city_name,y0,y1,p0,p1])
        
    
    if counter==1:
        df_year_pop=pd.DataFrame(year_pop,columns=['Country','City','from Year','to Year','from Population','to Population'])
        df_year_pop['Population Diff']=df_year_pop['to Population']-df_oldest['from Population']
        df_final=df_year_pop
    else:
        df_year_pop=pd.DataFrame(year_pop,columns=['Country','City','from Year','to Year','from Population','to Population'])
        df_year_pop['Population Diff']=df_year_pop['to Population']-df_oldest['from Population']
        df_final=pd.concat((df_final,df_year_pop),axis=0).reset_index(drop=True)
        
label=['Decrease','Stable','Increase']

## Defining bins width

margin=[(df_final['to Population']-df_final['from Population']).min()*1.1,-10000,10000,
        (df_final['to Population']-df_final['from Population']).max()*1.1]

## Decrease will be one with a Population Diff less than -10000 people
## Increase will be one with a Population Diff more than 10000 people
## Stable will be one with a Population Diff between -10000 and 10000 people

bins = pd.cut(df_final['Population Diff'],margin, labels=label)
final_final=df_final
final_final['Pop Eval']=bins
    
display(final_final)

,Country,City,from Year,to Year,from Population,to Population,Population Diff,Pop Eval
0,Iraq,Eridu,-3700,-3500,6000,10000,4000.0,Stable
1,Iraq,Larak,-3500,-3300,10000,10000,0.0,Stable
2,Iraq,Uruk,-3500,-3300,14000,40000,26000.0,Increase
3,Iraq,Nippur,-3000,-2800,10000,10000,0.0,Stable
4,Iraq,Adab,-2800,-2500,10000,20000,10000.0,Stable
5,Iraq,Kish,-2800,-2500,30000,20000,-10000.0,Decrease
6,Iraq,Larsa,-2800,-2500,16000,10000,-6000.0,Stable
7,Iraq,Shuruppak,-2800,-2500,30000,30000,0.0,Stable
8,Iraq,Ur,-2800,-2500,12000,10000,-2000.0,Stable
9,Iraq,Zabalam,-2800,-2500,10000,10000,0.0,Stable


CPU times: user 3.21 s, sys: 11.6 ms, total: 3.22 s
Wall time: 3.23 s


In [39]:
final_deviation=final_final['Pop Eval'].value_counts().sort_values()
Summ_pop_final=pd.DataFrame(np.r_[final_deviation,final_final['to Population'].sum()])
Summ_pop_final.columns=['Global']
Summ_pop_final.index=['Decrease','Increase','Stable','Total Pop']
display(Summ_pop_final)

,Global
Decrease,178
Increase,248
Stable,688
Total Pop,353610508


______